В этой статье мы поговорим об python-средах для решения kaggle-соревнований. Рассмотрим следующие вопросы:

1. **Локальный компьютер или облачные вычисления?** Какие преимущества у обоих подходов?
1. **Кратко о Jupyter и IPython** (которые лежат в основе Google Colab и Kaggle Kernel)
1. **Jupyter-ноутбуки против .py файлов**: плюсы и минусы обоих подходов
1. **Jupyter и Anaconda**. Как соотносятся и нужны ли друг для друга?
1. **Google Colab и Kaggle Kernel**: функциональность, ограничения по времени использования, сохранение и загрузка данных, версионирование кода, взаимодействие с Kaggle
1. **Система WSL для Windows** - для тех, кто хочет использовать локальный компьютер, но установлен Windows 10, а хочется Linux
1. **Сервис аренды GPU Vast.ai** - для тех, кто упирается в лимиты использования на Kaggle или в Colab, хочет больше вычислительных ресурсов и готов за них платить
1. **Облачное хранилище Google Cloud Storage** и его отличия от Google Drive

### Локальный компьютер или облачные вычисления?

Зачем нужны облачные вычисления?
1. Не у всех есть мощная видеокарта и процессор. Особенно это актуально для обучения нейронных сетей.
2. При использовании облачных вычислений код воспроизводим и им легко делиться. Вы копируете ссылку и пересылаете ее, другой человек ее открывает, запускает и получает тот же результат. Это дело буквально нескольких секунд, и это удобнее, чем пересылать код в виде файлов.
3. Меньше проблем с установкой библиотек. Я неоднократно встречал ситуацию, когда на локальном компьютере у человека что-то не работает, он переходит в Colab, и там все работает.

Но у локального компьютера тоже есть свои преимущества:
1. Нет необходимости работать с облачными хранилищами
2. Нет никаких ограничений по времени работы

### Кратко о возможностях Jupyter и IPython

Некогда люди решили, что очень удобно было бы работать с python-кодом в виде ячеек, которые можно запускать в любом порядке. Так возник Jupyter Notebook. Позже появилась идея о том, что Jupyter Notebook мог бы поддерживать и другие языки программирования. Проект разделился на два проекта: cобственно **Jupyter Notebook** и **IPython**. Мы не будем подробно рассматривать, чем они отличаются, потому что практически всегда используются в связке друг с другом. Google Colab и Kaggle Kernel также основаны на jupyter-ноутбуках.

Давайте кратко вспомним некоторые из возможностей Jupyter и IPython.

- Результат работы последней строки кода в ячейке автоматически выводится на печать. Чтобы резульат не выводился, нужно поставить в конце последней строки точку с запятой.
- Можно выделить несколько строк (или всю ячейку: `Ctrl-A`) и нажать сочетание клавиш `Ctrl+/`, чтобы закомментировать или раскомментировать все строки сразу.
- Вывод ячейки можно очищать командой clear_output:

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

Параметр `wait=True` означает, что нужно подождать с очисткой вывода до тех пор, пока не появится новый вывод.
- Выводом ячейки может быть не только текст, но и, например, рисунок matplotlib, произвольный HTML или интерактивный виджет:

<img src="assets/ipython.png" width="800" align="center">

- Можно выполнять любые команды операционной системы. Например в коде ниже мы создаем 3 директории внутри текущей, а затем проверяем содержимое текущей директории. В команды операционной системы можно передавать аргументы в фигурных скобках.

In [ ]:
for name in ['train', 'val', 'test']:
    !mkdir {name}
!ls

- Можно даже выполнить произвольную команду, заданную в виде строки:

In [ ]:
cmd = 'unzip -q -o train.zip'
!{cmd}

- Вывод команды можно записывать в переменную:

In [13]:
output = !ls
output

['sample_data  test  train  val']


Однако у такого способа запуска команд есть два недостатка. Во-первых такие команды работают только в IPython (хотя есть [способ](https://stackoverflow.com/a/58392953) запустить их и из скрипта .py, но синтаксис будет уже другим). Второй недостаток в том, что если вы в захотите прервать выполнение кода, и в это время выполнялась команда ОС, то эта команда прервется, однако выполнение ячейки при этом не остановится, а продолжится со следующей строки. Поэтому, например, запустив такую ячейку кода, остановить ее будет возможно только перезапуском среды исполнения, при этом все переменные среды будут потеряны.

In [ ]:
while True:
    !touch 123

- В IPython есть так называемые magic-команды, начинающиеся со знака процента. Самый важный пример - смена рабочей директории. Несмотря на то, что `cd` - это команда операционной системы, если ее записать с восклицательного знака, то по [некоторым причинам](https://stackoverflow.com/questions/36512527/cd-vs-cd-vs-cd-in-ipython) она не будет иметь эффекта. Нужно начинать ее со знака процента.

In [ ]:
%cd my_directory

- Есть еще одна полезная команда `%run`. Если после нее указать имя файла со скриптом, то это будет расносильно тому, как если бы вы скопировали содержимое этого файла в ячейку и запустили ее. То есть переменные, созданные внутри этот файла, будут доступны, и с ними можно будет продолжить работу в интерактивном режиме.

In [ ]:
!wget https://raw.githubusercontent.com/sedol1339/ml_utils/main/perform_tests.py
%run perform_tests.py

Это может быть полезно если у вас есть набор функций, которые вы часто используете. Вы записали их в файл, сохранили на Github, и когда создаете новый ноутбук - скачиваете этот файл с Github и выполняете, без необходимости копировать весь код в первую ячейку. Таким образом ваш ноутбук получается более лаконичным.

### Jupyter-ноутбуки против .py файлов

Преимущества Jupyter над .py скриптами понятны, но есть и недостатки:

- Большие проблемы с систематизацией кода. Ноутбуки разрастаются в размерах и становятся нечитаемыми.
- У Jupyter есть массивы In и Out, которые держат в памяти все входы и выходы ячеек, что может привести к переполнению памяти.
- Python-программист, всегда работавший с Jupyter, может не уметь работать с .py-скриптами, которые повсеместно используются в production.
- Вызывает привыкание к не очень хорошему code style: копирование ячеек кода вместо использования функций.

Более подробно можно почитать [здесь](https://docs.google.com/presentation/d/1n2RlMdmv1p25Xy5thJUhkKGvjtV-dkAIsUXP-AL4ffI/edit#slide=id.g362da58057_0_644).

<img src="assets/untitled.jpg" width="600" align="center">

### Jupyter и Anaconda

Anaconda - это программный пакет, содержащий интерпретатор python, виртуальные среды и Jupyter ноутбук. Однако я не рекомендовал бы начинающим python-программистам использовать Anaconda по нескольким причинам:

1. Основным преимуществом Anaconda является наличие нескольких независимых виртуальных сред. Но это же является и минусом, поскольку это запутывает новичков, которым почти никогда не требуется несколько независимых сред. Если же все-таки потребуется, то лучше разобраться с `venv`, это даст полезные навыки, которые могут пригодиться на работе.
1. Многие библиотеки предустановлены в Anaconda, но их без проблем можно установить и вручную через консоль. А тот, у кого они изначально были предстановлены, может так и не научиться устанавливать их вручную.
1. Использование Anaconda изрядно запутывает, когда у вас есть команды `!pip install` и `!conda install`, выполняющие почти одно и то же.
1. Работая только в Anaconda, вы можете так и не научиться использовать python без Anaconda, что неприятно удивит работодателя.
1. При использовании Anaconda могут возникнуть сложности с запуском Jupyter ноутбука из произвольной директории. Если же вы работаете без Anaconda, то Jupyter ноутбук устанавливается и запускается очень легко из любой директории.

Остановимся подробнее на последнем пункте. Вместо Jupyter я рекомендую использовать JupyterLab - более новую версию с большими возможностями. Покажу на примере Windows. Установив [интерпретатор python](https://www.python.org/downloads/), откройте консоль (shift+ПКМ, "открыть окно PowerShell здесь") и напишите: `pip install jupyterlab`. Затем в нужной вам директории откройте консоль и напишите: `python -m jupyterlab`, или создайте .bat файл с такой строкой и запустите его. Откроется страница в браузере, где вы сможете создавать и редактировать ноутбуки в данной директории. Это даже проще, чем в Anaconda.

Далее перейдем к обзору Google Colab.

### Google Colaboratory (Colab)

Colab - это не просто интерпретатор python, а полноценная виртуальная машина, работающая под управлением операционной системы Ubuntu.  В колабе можно делать практически что угодно: вы можете даже компилировать и исполнять код на C++ или на Java, если вы умеете это делать с помощью команд операционной системы. И, кстати, с помощью ячеек кода в Colab можно работать не только с языком Python, но и с языком R. В интернете можете найти инструкцию о том, как это сделать.

В настройках Colab вы можете выбрать аппаратный ускоритель, то есть видеокарту или тензорный процессор (последний, впрочем, используется довольно редко). В платной pro-версии можно выбрать также объем оперативной памяти.

Перезапуск среды исполнения перезапускает интерпретатор python, но при этом содержимое диска не очищается, и библиотеки, которые вы установили командой `!pip install ...`, не стираются. При сбросе среды исполнения виртуальная машина выключается и диск очищается.

Удаленные ячейки, как и в других Jupyter-средах, можно восстанавливать сочетанием клавиш `Ctrl+M+Z`.

Также доступен **менеджер файлов** (1). Здесь можно загружать файлы с локального компьютера и скачивать их. Однако загрузка файлов с вашего компьютера в Colab идет очень медленно. Если у вас большой файл, проще загрузить его через Google-диск, об этом чуть позже.

И, наконец, доступен **терминал** (2). Сюда можно вводить команды операционной системы даже в то время, пока ноутбук занят выполнением ячейки кода.

Через кнопку **Поделиться** (3) можно открыть доступ к ноутбуку по ссылке. Любой, кто откроет ссылку, сможет выполнить ваш код (при этом на вашу версию ноутбука это никак не повлияет). Либо вы можете открыть доступ к папке на гугл-диске, в которой лежит этот ноутбук.

<img src="assets/colab.jpg" width="800" align="center">

**Лимит времени**

Colab бывает трех версий - бесплатной, pro и pro+. Pro-версия стоит около 600 рублей в месяц. При этом будет доступна более мощная видеокарта и больший лимит времени. Например вы можете запустить расчет на ночь, а утром он у вас все еще будет работать и не выключится (и вы сможете продлжить с ним интерактивную работу). Хотя если посреди ночи расчет закончится, и видеокарта будет простаивать, тогда ноутбук может выключиться.

*Примечание. Есть и другие среды для облачных вычислений на GPU, например Gradient Community Notebooks, в котором тоже есть расширенная платная версия.*

**Cортировка и версионирование кода**

Историю версий ноутбука можно посмотреть в меню "файл", там можно сравнить разные версии и либо восстановить нужную версию, либо открыть ее в другом окне. Версии периодически создаются автоматически, но можно также создать версию вручную командой "сохранить и закрепить версию".

По умолчанию ноутбуки сохраняются на ваш Google Drive, и в меню "файл" есть кнопка "показать расположение на диске".

Если вы хотите создать в папке на Google Drive новый ноутбук, то нажимаете правой кнопкой мыши - "еще" - Google Colaboratory. Если этой кнопки нет, то нажмите "подключить другие приложения" и найдите там Google Colaboratory.

<img src="assets/colab2.jpg" width="650" align="center">

**Переиспользование кода**

Рассмотрим такую ситуацию, когда у вас несколько ноутбуков используют один и тот же набор функций. Здесь есть как минимум 3 варианта

1. *Хранить файл на Github*

Вы загружаете файл в любое место, откуда его можно скачать, например в репозиторий Github. В ноутбуке вы либо клонируете репозиторий, либо копируете конкретный файл, а затем выполняете его командой %run, как было показано в примере выше.

2. *Взять код из другого ноутбука*

Вы пишете нужные вам функции в отдельном ноутбуке, снабжая их заголовками. Дальше вы открываете настройки Colab и вставляете URL этого ноутбука в раздел "custom snippet notebook URL". Затем вы открываете любой ноутбук, нажимаете "фрагменты кода" и здесь вы видите все созданные вами сниппеты. Вы нажимаете, и сниппет копируется в ячейку. Вы его запускаете и затем можете эту ячейку удалить.

3. *Создать pip-пакет*

Есть гайды в интернете о том, как это сделать. После этого вы сможете выполнять нужный вам код командами `pip install` и `import`. То есть фактически вы можете создать собственную небольшую библиотеку для языка python.

**Взаимодействие с облачными хранилищами**

В Google Colab встроена возможность монтирования Google Drive. Делается это очень простым кодом, можно посмотреть [этот официальный пример](https://colab.research.google.com/notebooks/io.ipynb). После монтирования гугл-диск будет виден как папка gdrive. В нее можно писать файлы и с нее можно считывать файлы. Когда вы пытаетесь считать файл из этой папки, операционная система скачает его с гугл-диска. Но если вы собираетесь этим файлом часто пользоваться, то лучше его скопировать куда-то локально, например в папку /content.

Есть еще одно, более удобное облачное хранилище Google Cloud Storage, которое мы рассмотрим далее.

**Взаимодействие с Kaggle**

Для того, чтобы загрузить в Colab данные соревнований с сайта Kaggle, нужно сначала зайти в аккаунт на Kaggle и нажать "Create New API Token". Скачается файл kaggle.json. Заходим в Colab и загружаем этот файл. Затем выполняем следующий код. И после этого мы можем выполнить следующие команды:

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c <имя_соревнования>

Все данные соревнования будут скачаны в нашу рабочую директорию.

### Kaggle Kernel

Это еще одна python-среда для облачных вычислений, от сайта Kaggle. Новый ноутбук можно создать либо на странице соревнования, либо из раздела "Code" на главной странице.

Начнем с функциональности. Как и в Colab, можно выбрать аппаратный ускоритель (1). Есть такж кнопка включения в ноутбуке интернета (2) - это означает, что виртуальная машина получит доступ к сети. Если интернет в ноутбуке выключен, то вы не сможете запускать код, который скачивает что-то из интернета. Чтобы включить интернет или использовать видеокарту, нужно подтвердить номер телефона в настройках аккаунта.

Есть также раздел "environment" (3). В Kaggle, так же, как и в Colab, предустановлено много библиотек, и большинство из них постоянно обновляется. Выбрав "pin to original envormnment", вы отключите эти обновления для данного ноутбука. Таким образом вы сможете гарантировать (ну или почти гарантировать), что если ваш код работает сегодня, то он будет работать и завтра. Но вообще обновления библиотек редко что-то ломают, поэтому рекомендую ставить опцию "always use latest environment".

**Загрузка данных в ноутбук**

Теперь разберемся откуда брать данные. Загружать файлы с локального компьютера, к сожалению, нет возможности, но можно брать данные с платформы Kaggle или с Google Cloud Storage. Данные в Kaggle бывают двух видов: данные соревнований и датасеты. Датасеты, в свою очередь, делятся на публичные и приватные. Публичные датасеты можно найти в разделе "data". Приватные датасеты вы можете создать сами или запросить у кого-то доступ. В датасетах, несмотря на название, можно хранить не только обучающие данные, но и обученные нейросети или вообще что угодно.

Как же с этими данными работать? Когда вы запускаете ноутбук, в нем есть папка /kaggle/input (4). Если вы запустили ноутбук со страницы соревнования, то в папке /kaggle/input сразу же появятся данные этого соревнования. И вы можете добавить данные других соревнований или другие датасеты, к которым у вас есть доступ. Например если вы добавите этот датасет, то он появится у вас в папке /kaggle/input.

Из папки /kaggle/input можно считать файлы, но туда невозможно ничего сохранить. Куда же сохранять файлы? Для этого есть две папки: /kaggle/temp и /kaggle/working. Причем первая из них не существует, но ее можно создать. Папка /kaggle/temp предназначена для временных файлов, которые станут не нужны когда ноутбук отработает. Например, если данные соревнования представлены в виде архива, то их можно разархивировать в папку /kaggle/temp. Папка /kaggle/working (5) предназначена для тех файлов, которые являются результатами работы ноутбука. Как правило это файл для сабмита, то есть ваше предсказание, которое вы загружаете по кнопке "submit predictions" чтобы попасть в лидерборд. Кроме того это могут быть файлы моделей, которые вы хотите использовть в дальнейшем. То есть это любые файлы, которые вам понадобятся, когда ноутбук отработает.

<img src="assets/kaggle.jpg" width="800" align="center">

**Сессии**

Сессии в Kaggle kernel бывают двух видов: интерактивные сессии и бэкграунд-сессии. Если ваш ноутбук открыт во вкладке браузера, и вы можете запускать в нем ячейки - то значит это интерактивная сессия. Но проблема в том, что если вы запускаете расчет и уходите, то интерактивная сессия довольно быстро отключается (намного быстрее, чем в Colab) и вы теряете все данные. И в этом случае нужно использовать бэкграунд-сессии. Давайте пошагово разберемся как работать с бэкграунд-сессиями.

Бэкграунд-сессия - это запуск всего ноутбука с начала до конца на серверах Kaggle, причем *ваш компьютер при этом вообще может быть выключен*. Например, вы пишете код обучения модели, получения предсказаний и сохранения предсказаний в файл. И теперь вы хотите все это запустить. Здесь вам нужно внимательно следить, чтобы в папке /kaggle/working не оказалось больше 2 Гб данных, иначе эти данные будут стерты.

Приведу пример. Пусть вы обучаете модель распознавать автомобили на изображении. У вас есть архив с обучающими изображениями, который занимает 3 Гб. Если вы его разархивируете в папку /kaggle/working, и туда же запишете файл для сабмита, то когда ноутбук отработает все из папки /kaggle/working будет стерто, в том числе ваш файл для сабмита. Поэтому вам нужно разархивировать изображения в папку /kaggle/temp.

Теперь перейдем к тому, как же запустить бэкграунд-сессию. Для этого нужно нажать Save version и выбрать "save and run all (commit)". Далее нажать "advanced settings" и выбрать либо **"always save output"**, либо "save output for this version". И также выбрать использование видеокарты, если она вам нужна. После этого вам нужно нажать Save и можно закрыть страницу.

<img src="assets/kaggle2.jpg" width="800" align="center">

Бэкграунд-сессия будет работать в течение максимум 8 часов. Если ваш ноутбук не успеет отработать за это время. то его работа будет прервана. На любой странице Kaggle снизу слева есть кнопка **"view active events"**. Здесь мы видим нашу бэкграунд-сессию. Когжа она отработает, нажимаем на нее. Мы попадем на страницу, где  увидим выходя ячеек кода, а во вкладке Data появятся наши выходные данные. Здесь их можно либо скачать, либо сразу сделать сабмит в лидерборд.

**Взаимодействие с облачными хранилищами**

Здесь все довольно-таки непросто, потому что Kaggle можно взаимодействовать с Google Cloud Storage, но (на момент написания этой статьи) не умеет взаимодействовать в Google Drive. И если у вас нет хранилища в Google Cloud Storage, то каким же образом можно перенести промежуточные данные (например ваши не до конца обученные модели) с одной версии ноутбука на другую? Делается это так. Вы заходите в раздел Output ноутбука, находите там файлы моделей, которые вы сохраняли, и нажимаете New dataset. Таким образом вы сохраняете эти данные как датасет, после чего можно использовать этот датасет в любом вашем ноутбуке.

### Система WSL для Windows

Вместо облачных вычислений иногда удобно использовать локальный компьютер. Однако на компьютерах с Windows недоступны многие shell-команды, с которыми привыкли работать программисты. Есть простой выход из этой ситуации - установка системы WSL 2 (Windows Subsystem for Linux). Здесь я кратко опишу как настроить эту систему.

**Шаг 1.** Запускаем приложение Microsoft Store и ищем по запросу "WSL". Находим приложение "Ubuntu 18.04 LTS". Нажимаем "получить". Входить в аккаунт Microsoft не обязательно, когда предложит войти в аккаунт - закрываем окно. После установки нажимаем "запустить". Когда откроется окно, нажимаем "закрепить на панели задач". Придумываем логин и пароль, попадаем в консоль Ubuntu.

*Примечание. Возможно кому-то придется выполнить следующий шаг, описанный в инструкции:*

> *To use this feature, one first needs to use "Turn Windows features on or off" and select "Windows Subsystem for Linux", click OK, reboot, and use this app. The above step can also be performed using Administrator PowerShell prompt: `Enable-WindowsOptionalFeature -Online -FeatureName Microsoft-Windows-Subsystem-Linux`*

<img src="assets/wsl.jpg" width="1000" align="center">

**Шаг 2.** Устанавливаем pip и jupyter:

```
sudo apt update
sudo apt install python3-pip
pip3 install jupyter
```

Если что-то не работает - проверьте не закончилось ли место на диске C.

**Шаг 3.** Перезапускаем (закрываем окно и снова открываем) и запускаем jupyter:

```
jupyter notebook
```

Копируем URL из консоли в браузер и попадаем в Jupyter, где доступны команды linux. У WSL собственная файловая система (которая физически хранится на диске C). Работать в WSL с файлами, доступными из проводника Windows, можно через папку `/mnt`.

<img src="assets/wsl2.jpg" width="550" align="center">

### Сервис аренды GPU Vast.ai

Если вы хотите больше вычислительных ресурсов и большее время работы, то можно использовать сервисы аренды GPU, например [vast.ai](https://vast.ai/). После регистрации и привязки банковской карты вы попадаете на Marketplace.

<img src="assets/vastai.jpg" width="1000" align="center">

По кнопке "Edit image&config" вы модете выбрать Docker-образ для запуска, затем выбрать видеокарту (точнее, instance) и арендовать ее, нажав "Rent". Оплата на vast.ai почасовая, при этом не учитывается время, затрачиваемое на создание виртуальной машины. Снизить цену можно выбрав "Include Unverified Machines". Еще сильнее снизить цену можно выбрав "interruptiple", однако при этом работа виртуальной машины может быть прервана в любой момент. В этом нет ничего страшного если вы регулярно сохраняете данные. Более подробно можно почитать в разделе [FAQ](https://vast.ai/faq/). Конечно, при использовании vast.ai данные, которые вы загружаете в виртуальную машину, могут попасть в чужие руки, нужно это иметь в виду. Не исключено, что то же самое касается Colab и Kaggle.

###  Хранилище Google Cloud Storage

Google Cloud Storage (GCS) - это еще одно хранилище от компании Google, не связанное с Google Drive. Работа с GCS начинается с создания bucket'а - вашей "ячейки в хранилище". При этом ей надо придумать уникальное имя. Для создания bucket'а нужно указать данные банковской карты. Есть бесплатные лимиты использования, ппри превышении которых начнут списываться деньги.

Для хранилища можно установить различные права доступа. Мне показалось удобным разрешить скачивание со своего bucket'а без авторизации. Тогда я могу заходить в Colab и скачивать нужные мне данные одной командой, без необходимости авторизовываться в Google-аккаунт.

Скачивать данные можно тремя способами:

1. Через утилиту gsutil (`pip install gsutil`). В Colab она предустановлена. Пример:

```
!gsutil cp gs://oleg-zyablov/skillfactory/sf-dl-car-classification.zip .
```

2. Если данные сделаны публично доступными, то по URL, например:

```
!wget https://storage.googleapis.com/oleg-zyablov/skillfactory/sf-dl-car-classification.zip
```

3. Через сайт https://console.cloud.google.com/

Загружать данные можно двумя способами:

1. Через утилиту gsutil, при этом нужна авторизация (если только вы не разрешили в своем bucket'е загрузку без авторизации). Пример:

```
!gcloud auth login
!gsutil cp file.zip gs://oleg-zyablov/tmp/file.zip
```

2. Через сайт https://console.cloud.google.com/

Одним из преимуществ Google Cloud Storage является то, что с ним могут работать тензорные процессоры (TPU). Другим преимуществом является простота работы с хранилищем, по сравнению с Google Drive.